In [11]:
import nltk
from nltk.util import bigrams
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import numpy as np
import pandas as pd
from gensim import corpora, similarities, models
import streamlit as st

# Load your data from the Excel file
df = pd.read_excel("amazon_review_processed_full.xlsx")

# Define a function to preprocess and tokenize the text
def preprocess_text(row):
    # Combine the title and review columns and convert to lowercase
    title = row['Original title'] if not pd.isnull(row['Original title']) else ''
    review = row['Original review'] if not pd.isnull(row['Original review']) else ''
    text = (title + ' ' + review).lower()
    
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Tokenization into unigrams
    unigrams = word_tokenize(text)
    
    # Tokenization into bigrams
    bigrams_list = list(bigrams(unigrams))
    
    # Stemming
    stemmer = PorterStemmer()
    stemmed_unigrams = [stemmer.stem(word) for word in unigrams]
    stemmed_bigrams = [tuple(stemmer.stem(word) for word in bigram) for bigram in bigrams_list]
    
    # Define a set of stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove stopwords
    unigrams_without_stopwords = [word for word in stemmed_unigrams if word not in stop_words]
    
    # Convert bigrams to strings and remove stopwords
    bigrams_without_stopwords = [' '.join(bigram) for bigram in stemmed_bigrams if not any(word in stop_words for word in bigram)]
    
    # Join both unigrams and bigrams 
    tokens = unigrams_without_stopwords + bigrams_without_stopwords
    
    return tokens

# Apply the modified preprocessing function to your DataFrame
df['Processed Text'] = df.apply(preprocess_text, axis=1)
# Apply the preprocessing function to your DataFrame
df['Processed Text'] = df.apply(preprocess_text, axis=1)

# Create a dictionary from the tokenized content
dictionary = corpora.Dictionary(df['Processed Text'])

# Create a corpus (bag of words) from the tokenized content
corpus = [dictionary.doc2bow(text) for text in df['Processed Text']]

# To find the similarity scores, create a reverse index
Index = similarities.SparseMatrixSimilarity(corpus, len(dictionary))

# Create a TFIDF reverse index
TFIDF = models.TfidfModel(corpus)
corpus_TFIDF = [TFIDF[vec] for vec in corpus]
Index_TFIDF = similarities.SparseMatrixSimilarity(corpus_TFIDF, len(dictionary))

# A function to preprocess and tokenize the query
def preprocess_query(query):
    # Convert the query to lowercase
    query = query.lower()
    
    # Remove special characters and punctuation
    query = re.sub(r'[^\w\s]', ' ', query)
    
    # Tokenization into unigrams
    unigrams = word_tokenize(query)
    
    # Tokenization into bigrams
    bigrams_list = list(bigrams(unigrams))
    
    # Stemming
    stemmer = PorterStemmer()
    stemmed_unigrams = [stemmer.stem(word) for word in unigrams]
    stemmed_bigrams = [tuple(stemmer.stem(word) for word in bigram) for bigram in bigrams_list]
    
    # Define a set of stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove stopwords
    unigrams_without_stopwords = [word for word in stemmed_unigrams if word not in stop_words]
    bigrams_without_stopwords = [' '.join(bigram) for bigram in stemmed_bigrams if not any(word in stop_words for word in bigram)]
    
    # Join both unigrams and bigrams 
    tokens = unigrams_without_stopwords + bigrams_without_stopwords
    
    return tokens


# A function to search and sort the results based on a query
def search_and_sort(query):
    # Import nltk inside the app logic
    import nltk
    from nltk.tokenize import word_tokenize
    from nltk.stem import PorterStemmer
    nltk.download('punkt')
    
    # Preprocess the query and obtain a list of tokens
    preprocessed_query = preprocess_query(query)

    # Create a query vector using the same dictionary as the corpus
    qVector = dictionary.doc2bow(preprocessed_query)

    # Get its TFIDF from the same model as the corpus
    qVectorTFIDF = TFIDF[qVector]

    # Get the similarities from the two indexes (raw and TFIDF)
    simRaw = Index[qVector]
    simTFIDF = Index_TFIDF[qVectorTFIDF]

    # Create a DataFrame to store the results
    results_raw = pd.DataFrame({
        'Document': range(len(df)),
        'Raw Similarity Score': simRaw
    })

    results_tfidf = pd.DataFrame({
        'Document': range(len(df)),
        'TFIDF Similarity Score': simTFIDF
    })

    # Add the Similarity Score as a new column to the original DataFrame
    df['Raw Similarity Score'] = results_raw['Raw Similarity Score']
    df['TFIDF Similarity Score'] = results_tfidf['TFIDF Similarity Score']

    # Sort the DataFrame by TFIDF Similarity Score in descending order
    df_sorted_tfidf = df.sort_values(by='TFIDF Similarity Score', ascending=False)
    
    return df_sorted_tfidf[['Review Model', 'Review date', 'Review rating', 'Original title', 'Original review', 'TFIDF Similarity Score']].head(10)


In [10]:
df['Processed Text']

0        [work, great, wa, easi, instal, work, great, w...
1        [spunki, mid, size, printer, slower, print, sp...
2        [𝙲𝚕𝚎𝚊𝚛, 𝚙𝚛𝚒𝚗𝚝𝚜, 𝙻𝚘𝚟𝚎, 𝚒𝚝, 𝙲𝚕𝚎𝚊𝚛 𝚙𝚛𝚒𝚗𝚝𝚜, 𝚙𝚛𝚒𝚗𝚝𝚜...
3        [shaq, know, talk, yup, thi, printer, slam, du...
4        [user, friendli, would, think, someth, simpl, ...
                               ...                        
10204                    [quick, deliveri, quick deliveri]
10205                    [awesom, printer, awesom printer]
10206                                               [work]
10207    [el, artículo, llegó, roto, el artículo, artíc...
10208    [new, printer, work, like, one, use, veri, goo...
Name: Processed Text, Length: 10209, dtype: object

In [15]:
search_and_sort('low quality')

[nltk_data] Downloading package punkt to /Users/vannguyen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Review Model,Review date,Review rating,Original title,Original review,TFIDF Similarity Score
1593,HP ENVY 6055e,2021-12-03,3,Low Quality,this printer disconnects all the time. I spend...,0.392702
5238,Canon PIXMA TR4720,2023-08-16,2,Low quality printing,"It was easy to install, but the printing job s...",0.360529
7068,Epson - ET-4850,2023-05-14,1,Very Poor Quality,I have had this printer for several months. I...,0.265875
2076,Epson - ET-2800,2023-02-28,2,NaN,There’s only normal or high quality. There’s n...,0.262055
3767,Canon PIXMA TR4720,2023-06-07,1,Item had obviously been used. Ink cartridges w...,Printer did not produce quality copies. It arr...,0.238766
4819,Epson - XP-6100,2022-10-26,3,Could be better,Just taking it out of the box the printers pla...,0.230578
6853,HP ENVY Inspire 7255e,2023-03-29,3,Quality is poor unless you print a lot,I learned that you have to print often or else...,0.220677
4597,Epson - XP-6100,2023-08-20,1,Clunky,Low quality constructionSlow Print speedPainfu...,0.205049
2778,Epson - ET-3850,2023-03-07,3,Document Feeder is Low Quality,The ET-3850 is described as a home office prin...,0.199333
8810,HP OfficeJet Pro 9015e,2023-02-02,3,"Great printer, when it's ""online"".",Fast printer and great quality but I've had co...,0.194672
